# prepare OLLAMA

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [ ]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

>>> starting ollama serve


In [ ]:
# !ollama pull codellama
!ollama pull codellama:13b
# !ollama pull codellama:34b
# !ollama pull codellama:70b

time=2024-04-04T15:23:46.165Z level=INFO source=images.go:804 msg="total blobs: 0"
time=2024-04-04T15:23:46.166Z level=INFO source=images.go:811 msg="total unused blobs removed: 0"
time=2024-04-04T15:23:46.167Z level=INFO source=routes.go:1118 msg="Listening on 127.0.0.1:11434 (version 0.1.30)"
time=2024-04-04T15:23:46.167Z level=INFO source=payload_common.go:113 msg="Extracting dynamic libraries to /tmp/ollama2236437954/runners ..."
time=2024-04-04T15:23:55.125Z level=INFO source=payload_common.go:140 msg="Dynamic LLM libraries [cpu cpu_avx cuda_v11 rocm_v60000 cpu_avx2]"
time=2024-04-04T15:23:55.126Z level=INFO source=gpu.go:115 msg="Detecting GPU type"
time=2024-04-04T15:23:55.127Z level=INFO source=gpu.go:265 msg="Searching for GPU management library libcudart.so*"
time=2024-04-04T15:23:55.141Z level=INFO source=gpu.go:311 msg="Discovered GPU libraries: [/tmp/ollama2236437954/runners/cuda_v11/libcudart.so.11.0 /usr/local/cuda/lib64/libcudart.so.12.2.140]"
time=2024-04-04T15:23:55.2

# Prepare PandasAI

In [ ]:
!pip install -q 'pandasai[langchain]'

# Inference

In [ ]:
import pandas as pd
from pandasai import SmartDataframe
from langchain.llms import Ollama

llm = Ollama(model="codellama:13b")

In [ ]:
#@title Inference (auto-run) { run: "auto" }
# @markdown ## step 1
# @markdown Choose conversation
OPENDATA_OPTIONS = '15-per-hour-seattle' #@param ['15-per-hour-seattle', 'american-assembly.bowling-green', 'brexit-consensus', 'canadian-electoral-reform', 'football-concussions', 'march-on.operation-marchin-orders', 'scoop-hivemind.affordable-housing', 'scoop-hivemind.biodiversity', 'scoop-hivemind.freshwater', 'scoop-hivemind.taxes', 'scoop-hivemind.ubi', 'ssis.land-bank-farmland.2rumnecbeh.2021-08-01', 'vtaiwan.uberx']

url =  ('https://raw.githubusercontent.com/'
    'compdemocracy/openData/master/'
    f'{OPENDATA_OPTIONS}/comments.csv')
df = SmartDataframe(pd.read_csv(url, index_col=0), config={"llm": llm})

# @markdown ##step 2
# @markdown Requst of analysis or request for chart
request = Plot me a histogram of distribution of possitive, negative and neutral comments? #@param
df.chat(request)

Mounted at /content/drive
